In [ ]:
import numpy as np
from pprint import pprint
import pandas as pd
import os
import time

import gc
import random
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from keras.preprocessing import text, sequence

import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, MultiStepLR
import re

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold


In [ ]:
tqdm.pandas()

In [ ]:
class JigsawEvaluator(object):
    def __init__(self, y_true, y_identity, power=-5, overall_model_weight=0.25):
        self.y = (y_true >= 0.5).astype(int)
        self.y_i = (y_identity >= 0.5).astype(int)
        self.n_subgroups = self.y_i.shape[1]
        self.power = power
        self.overall_model_weight = overall_model_weight

    @staticmethod
    def _compute_auc(y_true, y_pred):
        try:
            return roc_auc_score(y_true, y_pred)
        except ValueError:
            return np.nan

    def _compute_subgroup_auc(self, i, y_pred):
        mask = self.y_i[:, i] == 1
        return self._compute_auc(self.y[mask], y_pred[mask])

    def _compute_bpsn_auc(self, i, y_pred):
        mask = self.y_i[:, i] + self.y == 1
        return self._compute_auc(self.y[mask], y_pred[mask])

    def _compute_bnsp_auc(self, i, y_pred):
        mask = self.y_i[:, i] + self.y != 1
        return self._compute_auc(self.y[mask], y_pred[mask])

    def compute_bias_metrics_for_model(self, y_pred):
        records = np.zeros((3, self.n_subgroups))
        for i in range(self.n_subgroups):
            records[0, i] = self._compute_subgroup_auc(i, y_pred)
            records[1, i] = self._compute_bpsn_auc(i, y_pred)
            records[2, i] = self._compute_bnsp_auc(i, y_pred)
        return records

    def _calculate_overall_auc(self, y_pred):
        return roc_auc_score(self.y, y_pred)

    def _power_mean(self, array):
        total = sum(np.power(array, self.power))
        return np.power(total / len(array), 1 / self.power)

    def get_final_metric(self, y_pred):
        bias_metrics = self.compute_bias_metrics_for_model(y_pred)
        bias_score = np.average([
            self._power_mean(bias_metrics[0]),
            self._power_mean(bias_metrics[1]),
            self._power_mean(bias_metrics[2])
        ])
        overall_score = self.overall_model_weight * self._calculate_overall_auc(y_pred)
        bias_score = (1 - self.overall_model_weight) * bias_score
        return overall_score + bias_score


In [ ]:
def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [ ]:
CRAWL_EMBEDDING_PATH = '../input/pickled-crawl300d2m-for-kernel-competitions/crawl-300d-2M.pkl'
GLOVE_EMBEDDING_PATH = '../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
PARA_EMBEDDING_PATH = '../input/pickled-paragram-300-vectors-sl999/paragram_300_sl999.pkl'

In [ ]:
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS

def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path,'rb') as f:
        emb_arr = pickle.load(f)
    return emb_arr

def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((max_features + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        if i <= max_features:
            try:
                embedding_matrix[i] = embedding_index[word]
            except KeyError:
                try:
                    embedding_matrix[i] = embedding_index[word.lower()]
                except KeyError:
                    try:
                        embedding_matrix[i] = embedding_index[word.title()]
                    except KeyError:
                        try:
                            embedding_matrix[i] = embedding_index[word.upper()]
                        except KeyError:
                            unknown_words.append(word)
    return embedding_matrix, unknown_words

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x
    
class NeuralNet(nn.Module):
    def __init__(self, embedding_matrix, num_aux_targets):
        super(NeuralNet, self).__init__()
        embed_size = embedding_matrix.shape[1]
        
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = SpatialDropout(0.3)
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)
        
        self.normal_linear = nn.Linear(ADD_FEATS_UNITS, DENSE_HIDDEN_UNITS)
    
        self.linear1 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        self.linear2 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        
        self.linear_out = nn.Linear(DENSE_HIDDEN_UNITS, 1)
        self.linear_aux_out = nn.Linear(DENSE_HIDDEN_UNITS, num_aux_targets) # num_aux_targets = 6 
        
    def forward(self, x, normal_feats, lengths=None):
        h_embedding = self.embedding(x.long())
        h_embedding = self.embedding_dropout(h_embedding)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        # global average pooling
        avg_pool = torch.mean(h_lstm2, 1)
        # global max pooling
        max_pool, _ = torch.max(h_lstm2, 1)
        
        h_conc = torch.cat((max_pool, avg_pool), 1)
        h_conc_linear1  = F.relu(self.linear1(h_conc))
        h_conc_linear2  = F.relu(self.linear2(h_conc))
        
        normal_linear  = F.relu(self.normal_linear(normal_feats.float()))
        
        hidden = h_conc + h_conc_linear1 + h_conc_linear2 + normal_linear
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1) # out = 7
        
        return out


In [ ]:
class SequenceBucketCollator():
    def __init__(self, choose_length, normal_feats_idx, sequence_index, length_index, label_index=None):
        self.choose_length = choose_length
        self.sequence_index = sequence_index
        self.length_index = length_index
        self.label_index = label_index
        self.normal_feats_idx = normal_feats_idx

    def __call__(self, batch):
        batch = [torch.stack(x) for x in list(zip(*batch))]
        
        sequences = batch[self.sequence_index]
        lengths = batch[self.length_index]
        normal_feats = batch[self.normal_feats_idx]

        length = self.choose_length(lengths)
        mask = torch.arange(start=maxlen, end=0, step=-1) < length
        padded_sequences = sequences[:, mask]
        
        batch[self.sequence_index] = padded_sequences
        batch[self.normal_feats_idx] = normal_feats
        
        if self.label_index is not None:
            return [x for i, x in enumerate(batch) if i != self.label_index], batch[self.label_index]
    
        return batch

In [ ]:
symbols_to_isolate = '.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ˈʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'
symbols_to_delete = '\n🍕\r🐵😑\xa0\ue014\t\uf818\uf04a\xad😢🐶️\uf0e0😜😎👊\u200b\u200e😁عدويهصقأناخلىبمغر😍💖💵Е👎😀😂\u202a\u202c🔥😄🏻💥ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ😋👏שלוםבי😱‼\x81エンジ故障\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘אעכח💩💯⛽🚄🏼ஜ😖ᴠ🚲‐😟😈💪🙏🎯🌹😇💔😡\x7f👌ἐὶήιὲκἀίῃἴξ🙄Ｈ😠\ufeff\u2028😉😤⛺🙂\u3000تحكسة👮💙فزط😏🍾🎉😞\u2008🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪\x08‑🐰🐇🐱🙆😨🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷улкнПоАН🐾🐕😆ה🔗🚽歌舞伎🙈😴🏿🤗🇺🇸мυтѕ⤵🏆🎃😩\u200a🌠🐟💫💰💎эпрд\x95🖐🙅⛲🍰🤐👆🙌\u2002💛🙁👀🙊🙉\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13🚬🤓\ue602😵άοόςέὸתמדףנרךצט😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓\uf0b7\uf04c\x9f\x10成都😣⏺😌🤑🌏😯ех😲Ἰᾶὁ💞🚓🔔📚🏀👐\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス😛外国人关系Сб💋💀🎄💜🤢َِьыгя不是\x9c\x9d🗑\u2005💃📣👿༼つ༽😰ḷЗз▱ц￼🤣卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003🚀🤴ʲшчИОРФДЯМюж😝🖑ὐύύ特殊作戦群щ💨圆明园קℐ🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅ﷻ🦄巨收赢得白鬼愤怒要买额ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟斤拷𝗢𝟳𝟱𝟬⦁マルハニチロ株式社⛷한국어ㄸㅓ니͜ʖ𝘿𝙔₵𝒩ℯ𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅ℴ𝒻𝒽𝓀𝓌𝒸𝓎𝙏ζ𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊多伦🐽🎻🎹⛓🏹🍷🦆为和中友谊祝贺与其想象对法如直接问用自己猜本传教士没积唯认识基督徒曾经让相信耶稣复活死怪他但当们聊些政治题时候战胜因圣把全堂结婚孩恐惧且栗谓这样还♾🎸🤕🤒⛑🎁批判检讨🏝🦁🙋😶쥐스탱트뤼도석유가격인상이경제황을렵게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용🔫👁凸ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡知彼百\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳点击查版🍭𝑥𝑦𝑧ＮＧ👣\uf020っ🏉ф💭🎥Ξ🐴👨🤳🦍\x0b🍩𝑯𝒒😗𝟐🏂👳🍗🕉🐲چی𝑮𝗕𝗴🍒ꜥⲣⲏ🐑⏰鉄リ事件ї💊「」\uf203\uf09a\uf222\ue608\uf202\uf099\uf469\ue607\uf410\ue600燻製シ虚偽屁理屈Г𝑩𝑰𝒀𝑺🌤𝗳𝗜𝗙𝗦𝗧🍊ὺἈἡχῖΛ⤏🇳𝒙ψՁմեռայինրւդձ冬至ὀ𝒁🔹🤚🍎𝑷🐂💅𝘬𝘱𝘸𝘷𝘐𝘭𝘓𝘖𝘹𝘲𝘫کΒώ💢ΜΟΝΑΕ🇱♲𝝈↴💒⊘Ȼ🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻𝐎𝐍𝐊𝑭🤖🎎😼🕷ｇｒｎｔｉｄｕｆｂｋ𝟰🇴🇭🇻🇲𝗞𝗭𝗘𝗤👼📉🍟🍦🌈🔭《🐊🐍\uf10aლڡ🐦\U0001f92f\U0001f92a🐡💳ἱ🙇𝗸𝗟𝗠𝗷🥜さようなら🔼'

In [ ]:
def count_regexp_occ(regexp="", text=None):
    """ Simple way to get the number of occurence of a regex"""
    # reference: https://www.kaggle.com/coolcoder22/lightgbm-fast-compact-solution
    return len(re.findall(regexp, text))

def normal_feature_engineering(text_list):
    add_feats_matrix = np.zeros((len(text_list), 11))
    
    for idx in range(len(text_list)):
        target_text = text_list[idx]
        # num words
        add_feats_matrix[idx, 0] = len(target_text.split(' '))
        # unique num words
        add_feats_matrix[idx, 1] = len(set(target_text.split(' ')))
        # url
        add_feats_matrix[idx, 2] = count_regexp_occ(r'http[s]{0,1}://\S+', target_text)
        # mail 
        add_feats_matrix[idx, 3] = count_regexp_occ(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', target_text)
        # all upper words
        add_feats_matrix[idx, 4] = count_regexp_occ(r'^[A-Z]+[A-Z]$', target_text)
        
        # symbols 1
        symbol_cnt = 0
        for symbol in symbols_to_isolate:
            if symbol in target_text:
                symbol_cnt += 1
        
        add_feats_matrix[idx, 5] = symbol_cnt
        
        # symbols 2
        symbol_delete_cnt = 0
        for symbol_del in symbols_to_delete:
            if symbol_del in target_text:
                symbol_delete_cnt += 1
        
        add_feats_matrix[idx, 6] = symbol_delete_cnt

        # rate features
        add_feats_matrix[idx, 7] = add_feats_matrix[idx, 1] / add_feats_matrix[idx, 0]
        add_feats_matrix[idx, 8] = add_feats_matrix[idx, 4] / add_feats_matrix[idx, 0]
        add_feats_matrix[idx, 9] = add_feats_matrix[idx, 5] / add_feats_matrix[idx, 0]
        add_feats_matrix[idx, 10] = add_feats_matrix[idx, 6] / add_feats_matrix[idx, 0]
    
    print('normal feature enginnering done: {}'.format(add_feats_matrix.shape))
    return add_feats_matrix

In [ ]:
def custom_loss(data, targets):
    ''' Define custom loss function for weighted BCE on 'target' column '''
    bce_loss_1 = nn.BCEWithLogitsLoss(weight=targets[:,1:2])(data[:,:1],targets[:,:1]) # 2번째 weight를 기반, 1번째 column이 input / target에서 1번째 칼럼이 아웃풋
    bce_loss_2 = nn.BCEWithLogitsLoss()(data[:,1:],targets[:,2:]) # output으로 내뱉는게 7개 칼럼
    return (bce_loss_1 * loss_weight) + bce_loss_2

In [ ]:
train = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

In [ ]:
import pickle

with open('../input/toxic2019-preprocessed/x_train.pkl', 'rb') as f:    
    x_train = pickle.load(f)

with open('../input/toxic2019-preprocessed/x_test.pkl', 'rb') as f:
    x_test = pickle.load(f)

In [ ]:
y_aux_train = train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]

identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
# Overall
weights = np.ones((len(x_train),)) / 4
# Subgroup
weights += (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4
# Background Positive, Subgroup Negative
weights += (( (train['target'].values>=0.5).astype(bool).astype(np.int) +
   (train[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
# Background Negative, Subgroup Positive
weights += (( (train['target'].values<0.5).astype(bool).astype(np.int) +
   (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
loss_weight = 1.0 / weights.mean()

y_train = np.vstack([(train['target'].values>=0.5).astype(np.int),weights]).T
y_train_torch = torch.tensor(np.hstack([y_train, y_aux_train]), dtype=torch.float32)

# for stratified kfold split
target_binary = (train['target'].values > 0.5).astype(int)

# saving for cv
y_true = (train['target'].values > 0.5).astype(int)
y_identity = train[identity_columns].values

max_features = 400000

In [ ]:
%%time
train_add_feats_matrix = normal_feature_engineering(train["comment_text"].values)
# test_add_feats_matrix = normal_feature_engineering(test["comment_text"].values)

ADD_FEATS_UNITS = train_add_feats_matrix.shape[1]
print("ADD_FEATS_UNITS: {}".format(ADD_FEATS_UNITS))

normal_feats = torch.from_numpy(train_add_feats_matrix)
# test_normal_feats = torch.from_numpy(test_add_feats_matrix)

del train_add_feats_matrix  # , test_add_feats_matrix
gc.collect()

In [ ]:
tokenizer = text.Tokenizer(num_words = max_features, filters='', lower=False)

tokenizer.fit_on_texts(list(x_train) + list(x_test))

In [ ]:
# para_matrix, unknown_words_para = build_matrix(tokenizer.word_index, PARA_EMBEDDING_PATH)
# print('n unknown words (para): ', len(unknown_words_para))

glove_matrix, unknown_words_glove = build_matrix(tokenizer.word_index, GLOVE_EMBEDDING_PATH)
print('n unknown words (glove): ', len(unknown_words_glove))

crawl_matrix, unknown_words_crawl = build_matrix(tokenizer.word_index, CRAWL_EMBEDDING_PATH)
print('n unknown words (crawl): ', len(unknown_words_crawl))

# embedding_matrix = para_matrix
# embedding_matrix = crawl_matrix*0.7 + glove_matrix*0.3
embedding_matrix = np.concatenate([crawl_matrix, glove_matrix], axis=-1)
embedding_matrix = np.concatenate([crawl_matrix, glove_matrix], axis=-1)
embedding_matrix = np.concatenate([para_matrix, crawl_matrix], axis=-1)
print(embedding_matrix.shape)

# del para_matrix, unknown_words_para
del glove_matrix, unknown_words_glove
del crawl_matrix, unknown_words_crawl
del train, test
gc.collect()

In [ ]:
# embedding_matrix = np.mean(np.array([crawl_matrix, glove_matrix]), axis=0)
# embedding_matrix.shape

# embedding_matrix = np.concatenate([crawl_matrix, glove_matrix], axis=-1)
# embedding_matrix.shape

In [ ]:
# n_embed = embedding_matrix.shape[1]
# embed_sampling = 400
# idx = np.random.permutation(n_embed)[:embed_sampling]
# embedding_matrix = embedding_matrix[:, idx]
# print(embedding_matrix.shape)

In [ ]:
x_train = tokenizer.texts_to_sequences(x_train)

lengths = torch.from_numpy(np.array([len(x) for x in x_train]))

maxlen = 220
# maxlen = lengths.max() 

x_train_padded = torch.from_numpy(sequence.pad_sequences(x_train, maxlen=maxlen))

del x_train
del x_test
del tokenizer
gc.collect()

In [ ]:
def train_model(train_loader, valid_loader, model, lr=0.001, 
                n_epochs=4, weight_file_name='fold.pt'):
    
    param_lrs = [{'params': param, 'lr': lr} for param in model.parameters()]
    optimizer = torch.optim.Adam(param_lrs, lr=lr)
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: 0.6 ** epoch)
    

    best_valid_loss = 1.0
    best_valid_fold = np.zeros((X_valid.size(0)))

    for epoch in range(n_epochs):
        start_time = time.time()
                
        model.train()
        avg_train_loss = 0
        
        for i, (x, y) in enumerate(train_loader):
            
            x[0] = x[0].cuda()
            x[1] = x[1].cuda()
            x[2] = x[2].cuda()
            y = y.cuda()
            
            y_pred = model(x[0], x[1], x[2])            
            loss = custom_loss(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            avg_train_loss += loss.item() / len(train_loader)
            
        avg_val_loss, valid_preds_fold = validation(model, valid_loader)
        
        elapsed = time.time() - start_time
        
        
        temp_dict = model.state_dict()
        del temp_dict['embedding.weight']
        torch.save(temp_dict, weight_file_name+str(epoch))
            
        if avg_val_loss < best_valid_loss:
            best_epoch = epoch + 1
            best_valid_loss = avg_val_loss   
            best_valid_fold = valid_preds_fold
            
        print("Epoch {} - avg_train_loss: {:.4f}  avg_val_loss: {:.4f}  lr: {:0.6f}  file: {}  time: {:.0f}s".format(
            epoch+1, avg_train_loss, avg_val_loss, scheduler.get_lr()[0], weight_file_name+str(epoch+1), elapsed))

        scheduler.step()


    return best_valid_fold, {
        'best_epoch': best_epoch,
        'best_loss': best_valid_loss,
        'weight_file_name' : weight_file_name,
    }

In [ ]:
def validation(model, valid_loader):
    
    model.eval()
    valid_preds_fold = np.zeros((X_valid.size(0)))
    avg_val_loss = 0.
    
    with torch.no_grad():
        for i, (x, y) in enumerate(valid_loader):
            x[0] = x[0].cuda()
            x[1] = x[1].cuda()
            x[2] = x[2].cuda()
            y = y.cuda()
            
            y_pred = model(x[0], x[1], x[2]).detach()
            avg_val_loss += custom_loss(y_pred, y).item() / len(valid_loader)
            valid_preds_fold[i * batch_size:(i + 1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]
        
    return avg_val_loss, valid_preds_fold

In [ ]:
batch_size = 512

splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=1234).split(x_train_padded, target_binary))

for seed in [90, 100]:

    print("===== {} seed training starts ====== ".format(seed))
    seed_everything(seed)

    train_preds = np.zeros((len(x_train_padded)))
    
    for fold in [0,1,2,3,4]:

        print("{0}/5 fold training starts!".format(fold+1))

        fold_num = str(fold + 1)

        trn_index, val_index = splits[fold]

        X_train, X_valid = x_train_padded[trn_index], x_train_padded[val_index]
        train_normal_feats, val_normal_feats = normal_feats[trn_index], normal_feats[val_index]
        train_length, valid_length = lengths[trn_index], lengths[val_index]
        Y_train, Y_valid = y_train_torch[trn_index], y_train_torch[val_index]


        train_dataset = data.TensorDataset(X_train, train_normal_feats, train_length, Y_train)
        valid_dataset = data.TensorDataset(X_valid, val_normal_feats, valid_length, Y_valid)


        train_collator = SequenceBucketCollator(lambda lengths: lengths.max(), sequence_index=0, length_index=2, normal_feats_idx=1, label_index=3)

        train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_collator)
        valid_loader = data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=train_collator)
        
        model = NeuralNet(embedding_matrix, y_aux_train.shape[-1]) # 6
        model.cuda()
        best_valid_fold, result = train_model(train_loader, valid_loader, model, lr=0.001, n_epochs=6, weight_file_name=str(seed)+'_' + fold_num + 'fold.pt')
        print(result)

        train_preds[val_index] = best_valid_fold
        
        del model, train_loader, valid_loader, best_valid_fold
        gc.collect()

    evaluator = JigsawEvaluator(y_true, y_identity)
    auc_score = evaluator.get_final_metric(train_preds)
    print(f'cv score: {auc_score:<8.5f}')
    print()
